In [117]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

print('numpy version : ',np.__version__)
print('pandas version : ',pd.__version__)
print('seaborn version : ',sns.__version__)

numpy version :  1.20.1
pandas version :  1.2.4
seaborn version :  0.11.1


In [118]:
df = pd.read_csv('credit-g_csv.csv')

In [119]:
def segment(x):
    if x['class'] == 'good':
        class_encoded = '0'
    else:
        class_encoded = '1'
    return class_encoded

df['class'] = df.apply(lambda x: segment(x), axis=1)

In [120]:
df["class"] = pd.to_numeric(df["class"], downcast="float")

In [121]:
df_class = df['class']
df_class

0      0.0
1      1.0
2      0.0
3      0.0
4      1.0
      ... 
995    0.0
996    0.0
997    0.0
998    1.0
999    0.0
Name: class, Length: 1000, dtype: float32

In [122]:
def calc_iv(df, feature, target, pr=False):

    d1 = df.groupby(by=feature, as_index=True)
    data = pd.DataFrame()

    data['all'] = d1[target].count()
    data['bad'] = d1[target].sum()
    data['good'] = data['all']-data['bad']
    data['share'] = data['all'] / data['all'].sum()
    data['bad_rate'] = d1[target].mean()
    data['d_g'] = (data['all'] - data['bad']) / (data['all'] - data['bad']).sum()
    data['d_b'] = data['bad'] / data['bad'].sum()
    data['woe'] = np.log(data['d_g'] / data['d_b'])
    data = data.replace({'woe': {np.inf: 0, -np.inf: 0}})
    data['iv'] = data['woe'] * (data['d_g'] - data['d_b'])

    data.insert(0, 'variable', feature)
    data.insert(1, 'value', data.index)
    data.index = range(len(data))

    iv = data['iv'].sum()

    if pr:
        print(data)
        print('IV = %s' % iv)

    return iv, data

In [124]:
df['duration'] = pd.qcut(df['duration'], q=7)

In [127]:
iv, data = calc_iv(df, 'housing', 'class')

In [128]:
data

,variable,value,all,bad,good,share,bad_rate,d_g,d_b,woe,iv
0,housing,for free,108,44.0,64.0,0.108,0.407407,0.091429,0.146667,-0.472604,0.026106
1,housing,own,713,186.0,527.0,0.713,0.260870,0.752857,0.620000,0.194156,0.025795
2,housing,rent,179,70.0,109.0,0.179,0.391061,0.155714,0.233333,-0.404445,0.031393


In [34]:
data['bad_rate'].sort_values(ascending = True)

6     0.180000
14    0.183673
5     0.200000
10    0.215686
12    0.220000
13    0.235294
0     0.240000
7     0.260000
11    0.265306
9     0.280000
2     0.285714
3     0.313726
8     0.320000
16    0.320000
4     0.326531
18    0.360000
1     0.372549
15    0.400000
17    0.440000
19    0.580000
Name: bad_rate, dtype: float32

In [73]:
dfa = df[df['credit_amount'] <= 708.95 ]

In [91]:
def segment(x):
    if x['class'] == 'good':
        class_encoded = '0'
    else:
        class_encoded = '1'
    return class_encoded

df['class'] = df.apply(lambda x: segment(x), axis=1)

In [92]:
df["class"] = pd.to_numeric(df["class"], downcast="float")

In [76]:
def calc_iv(df, feature, target, pr=False):

    d1 = df.groupby(by=feature, as_index=True)
    data = pd.DataFrame()

    data['all'] = d1[target].count()
    data['bad'] = d1[target].sum()
    data['good'] = data['all']-data['bad']
    data['share'] = data['all'] / data['all'].sum()
    data['bad_rate'] = d1[target].mean()
    data['d_g'] = (data['all'] - data['bad']) / (data['all'] - data['bad']).sum()
    data['d_b'] = data['bad'] / data['bad'].sum()
    data['woe'] = np.log(data['d_g'] / data['d_b'])
    data = data.replace({'woe': {np.inf: 0, -np.inf: 0}})
    data['iv'] = data['woe'] * (data['d_g'] - data['d_b'])

    data.insert(0, 'variable', feature)
    data.insert(1, 'value', data.index)
    data.index = range(len(data))

    iv = data['iv'].sum()

    if pr:
        print(data)
        print('IV = %s' % iv)

    return iv, data

In [59]:
iv, data = calc_iv(df, 'credit_amount', 'class')

In [61]:
iv

0.2002607862277774

In [94]:
dfb = dfb[dfb['credit_amount'] <= 932 ]

In [95]:
iv, data = calc_iv(dfb, 'credit_amount', 'class')

In [136]:
data['bad_rate']

0     0.0
1     0.3
2     0.2
3     0.3
4     0.4
     ... 
95    0.5
96    0.3
97    0.7
98    0.6
99    0.8
Name: bad_rate, Length: 100, dtype: float32

In [89]:
df_train_oversample = df

In [301]:
X = df_train_oversample
y = df_train_oversample['class']

In [88]:
from xverse.transformer import MonotonicBinning

clf = MonotonicBinning()
clf.fit(X, y)

print(clf.bins)
output_bins = clf.bins

NameError: name 'X' is not defined

In [236]:
clf = MonotonicBinning(custom_binning=output_bins) #output_bins was created earlier

out_X = clf.transform(X)
out_X.head()

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,"(3.999, 12.0]",critical/other existing credit,radio/tv,"(249.999, 1554.0]",no known savings,>=7,4,male single,none,...,real estate,"(38.0, 75.0]",none,own,2,skilled,1,yes,yes,0.0
1,0<=X<200,"(24.0, 72.0]",existing paid,radio/tv,"(3368.0, 18424.0]",<100,1<=X<4,2,female div/dep/mar,none,...,real estate,"(18.999, 28.0]",none,own,1,skilled,1,none,yes,1.0
2,no checking,"(3.999, 12.0]",critical/other existing credit,education,"(1554.0, 3368.0]",<100,4<=X<7,2,male single,none,...,real estate,"(38.0, 75.0]",none,own,1,unskilled resident,2,none,yes,0.0
3,<0,"(24.0, 72.0]",existing paid,furniture/equipment,"(3368.0, 18424.0]",<100,4<=X<7,2,male single,guarantor,...,life insurance,"(38.0, 75.0]",none,for free,1,skilled,2,none,yes,0.0
4,<0,"(12.0, 24.0]",delayed previously,new car,"(3368.0, 18424.0]",<100,1<=X<4,3,male single,none,...,no known property,"(38.0, 75.0]",none,for free,2,skilled,2,none,yes,1.0


In [237]:
from xverse.transformer import WOE
clf = WOE()
clf.fit(X, y)
clf.woe_df # weight of evidence transformation dataset. This dataset will be used in making bivariate charts as well. 
clf.iv_df #information value datase

,Variable_Name,Information_Value
1,checking_status,0.666011
4,credit_history,0.293234
20,savings_status,0.196010
18,purpose,0.169195
5,duration,0.168117
17,property_magnitude,0.112638
6,employment,0.086434
9,housing,0.083293
14,other_payment_plans,0.057615
3,credit_amount,0.048772


In [256]:
clf.woe_df[clf.woe_df['Variable_Name'] == 'credit_amount']

,Variable_Name,Category,Count,Event,Non_Event,Event_Rate,Non_Event_Rate,Event_Distribution,Non_Event_Distribution,WOE,Information_Value
9,credit_amount,"(249.999, 1554.0]",334,95.0,239.0,0.284431,0.715569,0.316667,0.341429,-0.075289,0.048772
10,credit_amount,"(1554.0, 3368.0]",333,84.0,249.0,0.252252,0.747748,0.280000,0.355714,-0.239338,0.048772
11,credit_amount,"(3368.0, 18424.0]",333,121.0,212.0,0.363363,0.636637,0.403333,0.302857,0.286502,0.048772


In [288]:
df_train = df

In [292]:
msk = df_train['class'] == 1
num_to_oversample = len(df_train) - 2*msk.sum()
df_positive_oversample = df_train[msk].sample(n=num_to_oversample, replace=True, random_state=888)
df_train_oversample = pd.concat([df_train, df_positive_oversample])
df_train_oversample['class'].value_counts()

0.0    700
1.0    700
Name: class, dtype: int64

In [293]:
df_train_oversample

,checking_status,duration,credit_history,purpose,credit_amount,savings_status,employment,installment_commitment,personal_status,other_parties,...,property_magnitude,age,other_payment_plans,housing,existing_credits,job,num_dependents,own_telephone,foreign_worker,class
0,<0,6,critical/other existing credit,radio/tv,1169,no known savings,>=7,4,male single,none,...,real estate,67,none,own,2,skilled,1,yes,yes,0.0
1,0<=X<200,48,existing paid,radio/tv,5951,<100,1<=X<4,2,female div/dep/mar,none,...,real estate,22,none,own,1,skilled,1,none,yes,1.0
2,no checking,12,critical/other existing credit,education,2096,<100,4<=X<7,2,male single,none,...,real estate,49,none,own,1,unskilled resident,2,none,yes,0.0
3,<0,42,existing paid,furniture/equipment,7882,<100,4<=X<7,2,male single,guarantor,...,life insurance,45,none,for free,1,skilled,2,none,yes,0.0
4,<0,24,delayed previously,new car,4870,<100,1<=X<4,3,male single,none,...,no known property,53,none,for free,2,skilled,2,none,yes,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35,0<=X<200,45,critical/other existing credit,radio/tv,4746,<100,<1,4,male single,none,...,life insurance,25,none,own,2,unskilled resident,1,none,yes,1.0
557,no checking,21,no credits/all paid,new car,5003,no known savings,1<=X<4,1,female div/dep/mar,none,...,life insurance,29,bank,own,2,skilled,1,yes,yes,1.0
983,<0,36,existing paid,used car,8229,<100,1<=X<4,2,male single,none,...,life insurance,26,none,own,1,skilled,2,none,yes,1.0
174,<0,21,delayed previously,education,3414,<100,<1,2,male single,none,...,life insurance,26,none,own,2,skilled,1,none,yes,1.0


In [276]:
def calc_iv(df, feature, target, pr=False):

    d1 = df.groupby(by=feature, as_index=True)
    data = pd.DataFrame()

    data['all'] = d1[target].count()
    data['bad'] = d1[target].sum()
    data['good'] = data['all']-data['bad']
    data['share'] = data['all'] / data['all'].sum()
    data['bad_rate'] = d1[target].mean()
    data['d_g'] = (data['all'] - data['bad']) / (data['all'] - data['bad']).sum()
    data['d_b'] = data['bad'] / data['bad'].sum()
    data['woe'] = np.log(data['d_g'] / data['d_b'])
    data = data.replace({'woe': {np.inf: 0, -np.inf: 0}})
    data['iv'] = data['woe'] * (data['d_g'] - data['d_b'])

    data.insert(0, 'variable', feature)
    data.insert(1, 'value', data.index)
    data.index = range(len(data))

    iv = data['iv'].sum()

    if pr:
        print(data)
        print('IV = %s' % iv)

    return iv, data

In [278]:
df_train_oversample['credit_amount'] = pd.qcut(df_train_oversample['credit_amount'], q=20)

In [279]:
iv, data = calc_iv(df_train_oversample, 'credit_amount', 'class')

In [280]:
data

,variable,value,all,bad,good,share,bad_rate,d_g,d_b,woe,iv
0,credit_amount,"(249.999, 709.0]",72,33.0,39.0,0.051429,0.458333,0.055714,0.047143,1.670541e-01,1.431893e-03
1,credit_amount,"(709.0, 929.9]",68,40.0,28.0,0.048571,0.588235,0.040000,0.057143,-3.566749e-01,6.114428e-03
2,credit_amount,"(929.9, 1183.5]",70,25.0,45.0,0.050000,0.357143,0.064286,0.035714,5.877866e-01,1.679390e-02
3,credit_amount,"(1183.5, 1275.0]",73,42.0,31.0,0.052143,0.575342,0.044286,0.060000,-3.036824e-01,4.772151e-03
4,credit_amount,"(1275.0, 1372.0]",68,38.0,30.0,0.048571,0.558824,0.042857,0.054286,-2.363887e-01,2.701585e-03
5,credit_amount,"(1372.0, 1512.0]",71,23.0,48.0,0.050714,0.323944,0.068571,0.032857,7.357068e-01,2.627524e-02
6,credit_amount,"(1512.0, 1738.6]",68,27.0,41.0,0.048571,0.397059,0.058571,0.038571,4.177352e-01,8.354704e-03
7,credit_amount,"(1738.6, 1935.0]",71,35.0,36.0,0.050714,0.492958,0.051429,0.050000,2.817086e-02,4.024407e-05
8,credit_amount,"(1935.0, 2145.55]",69,30.0,39.0,0.049286,0.434783,0.055714,0.042857,2.623642e-01,3.373254e-03
9,credit_amount,"(2145.55, 2359.5]",70,34.0,36.0,0.050000,0.485714,0.051429,0.048571,5.715838e-02,1.633096e-04


In [29]:
def woe_discrete(df, discrete_variable_name, good_bad_variable_df):
    df = pd.concat([df[discrete_variable_name], good_bad_variable_df], axis = 1)
    df = pd.concat([df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].count(),
                df.groupby(df.columns.values[0], as_index = False)[df.columns.values[1]].mean()], axis = 1)
    df = df.iloc[:, [0, 1, 3]]
    df.columns = [df.columns.values[0], 'n_obs', 'prop_good']
    df['prop_n_obs'] = df['n_obs'] / df['n_obs'].sum()
    df['n_good'] = df['prop_good'] * df['n_obs']
    df['n_bad'] = (1 - df['prop_good']) * df['n_obs']
    df['prop_n_good'] = df['n_good'] / df['n_good'].sum()
    df['prop_n_bad'] = df['n_bad'] / df['n_bad'].sum()
    df['WoE'] = np.log(df['prop_n_good'] / df['prop_n_bad'])
    df = df.sort_values(['WoE'])
    df = df.reset_index(drop = True)
    df['diff_prop_good'] = df['prop_good'].diff().abs()
    df['diff_WoE'] = df['WoE'].diff().abs()
    df['IV'] = (df['prop_n_good'] - df['prop_n_bad']) * df['WoE']
    df['IV'] = df['IV'].sum()
    return df

In [33]:
df_temp = woe_discrete(df, 'credit_amount', df_class)

In [34]:
df_temp

,credit_amount,n_obs,prop_good,prop_n_obs,n_good,n_bad,prop_n_good,prop_n_bad,WoE,diff_prop_good,diff_WoE,IV
0,"(9162.7, 18424.0]",50,0.420000,0.050,20.999999,29.000002,0.030000,0.096667,-1.170071,NaN,NaN,0.200261
1,"(5969.95, 7179.4]",50,0.560000,0.050,28.000000,22.000000,0.040000,0.073333,-0.606136,0.140000,0.563936,0.200261
2,"(3972.25, 4720.0]",50,0.600000,0.050,30.000001,19.999999,0.042857,0.066667,-0.441833,0.040000,0.164303,0.200261
3,"(708.95, 932.0]",51,0.627451,0.051,32.000001,18.999999,0.045714,0.063333,-0.326001,0.027451,0.115832,0.200261
4,"(7179.4, 9162.7]",50,0.640000,0.050,31.999999,18.000001,0.045714,0.060000,-0.271934,0.012549,0.054067,0.200261
5,"(1262.0, 1365.5]",49,0.673469,0.049,32.999999,16.000001,0.047143,0.053333,-0.123379,0.033469,0.148555,0.200261
6,"(4720.0, 5969.95]",50,0.680000,0.050,34.000000,16.000000,0.048571,0.053333,-0.093526,0.006531,0.029853,0.200261
7,"(1906.8, 2100.55]",50,0.680000,0.050,34.000000,16.000000,0.048571,0.053333,-0.093526,0.000000,0.000000,0.200261
8,"(1157.55, 1262.0]",51,0.686275,0.051,35.000001,15.999999,0.050000,0.053333,-0.064538,0.006275,0.028988,0.200261
9,"(932.0, 1157.55]",49,0.714286,0.049,35.000001,13.999999,0.050000,0.046667,0.068993,0.028011,0.133531,0.200261
